# Integrated Test
In this test, we'll use a SageMaker Estimator (https://sagemaker.readthedocs.io/en/stable/estimators.html) to encapsulate the docker image published to ECR and start a **local** test, but this time, using the SageMaker library.

In [ ]:
import sagemaker
import json
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix='mlops/iris'

## Upload the dataset
In the previous exercise, prepared the training and validation dataset. Now, we'll upload the CSVs to S3 and share them with an Estimator

In [ ]:
train_path = sagemaker_session.upload_data(path='input/data/training', key_prefix='iris-model/input/training')
test_path = sagemaker_session.upload_data(path='input/data/testing', key_prefix='iris-model/input/testing')
print("Training: %s\nTesting: %s" % (train_path, test_path) )

## And now, we can use a SageMaker Estimator for training and deploying the container we've created

In [ ]:
# Create the estimator
# iris-model:test is the name of the container created in the previous notebook
# By the local codebuild test. An image with that name:tag was pushed to the ECR.
iris = sagemaker.estimator.Estimator('iris-model:test',
                                    role,
                                    train_instance_count=1, 
                                    train_instance_type='local',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix))
hyperparameters = {
    'max_depth': 20,
    'n_jobs': 4,
    'n_estimators': 120
}

print(hyperparameters)
iris.set_hyperparameters(**hyperparameters)

After you call .fit, a new training job will be executed inside the *local Docker daemon* and not in the SageMaker environment, on the cloud

In [ ]:
iris.fit({'training': train_path, 'testing': test_path })

The next command will launch a new container in your local Docker daemon. Then you can use the returned predictor for testing it

In [ ]:
iris_predictor = iris.deploy(initial_instance_count=1, instance_type='local')

Now, let's use the predictor (https://sagemaker.readthedocs.io/en/stable/predictors.html) for some tests.

In [ ]:
import pandas as pd
import random
from sagemaker.predictor import csv_serializer, csv_deserializer

# configure the predictor to do everything for us
iris_predictor.content_type = 'text/csv'
iris_predictor.accept = 'text/csv'
iris_predictor.serializer = csv_serializer
iris_predictor.deserializer = None

# load the testing data from the validation csv
validation = pd.read_csv('input/data/testing/testing.csv', header=None)
idx = random.randint(0,len(validation)-5)
req = validation.iloc[idx:idx+5].values

# cut a sample with 5 lines from our dataset and then split the label from the features.
X = req[:,1:].tolist()
y = req[:,0].tolist()

# call the local endpoint
for features,label in zip(X,y):
    prediction = float(iris_predictor.predict(features).decode('utf-8').strip())

    # compare the results
    print("RESULT: {} == {} ? {}".format( label, prediction, label == prediction ) )

In [ ]:
iris.delete_endpoint()

### That's it! :) Now you can go back to the previous Jupyter notebook and commit the assets to start building the Final Docker Image